<a href="https://colab.research.google.com/github/MatInGit/Tensorflow-to-Tensorflow-Micro-converter/blob/main/TF_to_tf_Micro_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tensorflow to Tensorflow Micro converter**

This Notebook is based on the one available at [here](https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/hello_world/train/train_hello_world_model.ipynb#scrollTo=gD60bE8cXQId)

The objective is to streamline the process.

To start, drop your model on the left hand side (expand if needed) so that it is uploaded to the temporary storage.

In [ ]:
# ! pip install tensorflow==2.4.0
# ! pip install h5py
# ! pip install tensorflow_model_optimization # If you used quantization aware training 

import tensorflow as tf
# import tensorflow_model_optimization as tfmot
import numpy as np

# upload by dragging onto the left hand side panel

In [ ]:
model = tf.keras.models.load_model('<yout model name here>.h5')


# # If you used quantization aware training 
# with tfmot.quantization.keras.quantize_scope():
#     model = tf.keras.models.load_model('<yout model name here>.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]


# Enforce integer only quantization - Optional

# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8] 


# Generate Representative dataset - IMPORTANT

def representative_dataset(samples = 6400000):
    for _ in range(samples):                                                    # Change - Rule of thumb 1000*input size # warning this may take a while
      data = (np.random.rand(640,)*140) -60                                     # Adjust to your range or load a dataset
      yield [data.astype(np.float32)]                                           # Change to your input dtype


# This converts the model to tf lite
converter.representative_dataset = representative_dataset
tflite_model = converter.convert()

 
open("converted_model.tflite", "wb").write(tflite_model)


# This dumps the files as a hex file and saves it as a header file
!apt-get -qq install xxd
!echo "const unsigned char model[] = {" > /content/model.h
!cat converted_model.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h


import os
model_h_size = os.path.getsize("model.h")
print(f"Model size: {model_h_size:,} bytes, remmeber to edit the .h file.")